***Requirements***



*   openai>=1.40.0
*   anthropic>=0.34.0
*   google-genai>=0.3.0









In [ ]:
#!/usr/bin/env python3
# Env vars: OPENAI_API_KEY, ANTHROPIC_API_KEY, GEMINI_API_KEY, DEEPSEEK_API_KEY
import os, sys

if len(sys.argv) < 2:
    print("Usage: python run_gloss_llms_min.py \"Context and options text here\"")
    sys.exit(1)

INSTRUCTION_PROMPT = (
    "<Your task is to choose **only one** correct\n"
    "gloss that best matches the sense of the\n"
    "word in the given context. Please respond\n"
    "with **just the number** of the correct an-\n"
    "swer (e.g., ‘0‘, ‘1‘, ‘2‘, ...). Do not include any\n"
    "text, explanation, or justification.>"
)

task_text = sys.argv[1]
full_prompt = f"{INSTRUCTION_PROMPT}\n\n{task_text}"

# ---------- OpenAI: GPT-4 (gpt-4o) ----------
from openai import OpenAI as _OpenAI
_openai_client = _OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
_openai_resp = _openai_client.chat.completions.create(
    model="chatgpt-4o-latest",
    messages=[
        {"role": "system", "content": "Output only a single number (e.g., 0, 1, 2)."},
        {"role": "user", "content": full_prompt}
    ]
)
openai_answer = _openai_resp.choices[0].message.content.strip()

# ---------- Anthropic: Claude 3.5 Sonnet (October 2024 snapshot) ----------
import anthropic as _anthropic
_anthropic_client = _anthropic.Anthropic(api_key=os.environ.get("ANTHROPIC_API_KEY"))
_anthropic_msg = _anthropic_client.messages.create(
    model="claude-3-5-sonnet-20241022",

    messages=[{"role": "user", "content": full_prompt}],
)
claude_answer = "".join(getattr(p, "text", "") for p in _anthropic_msg.content).strip()

# ---------- Google: Gemini 1.5 Pro ----------
from google import genai as _genai
_gem_client = _genai.Client(api_key=os.environ.get("GEMINI_API_KEY"))
_gem_resp = _gem_client.models.generate_content(
    model="gemini-1.5-pro-latest",
    contents=full_prompt,
    config={"temperature": 0, "max_output_tokens": 8},
)
gemini_answer = (getattr(_gem_resp, "text", "") or "").strip()

# ---------- DeepSeek: V3 via OpenAI-compatible endpoint ----------
from openai import OpenAI as _DSOpenAI
_ds_client = _DSOpenAI(
    api_key=os.environ.get("DEEPSEEK_API_KEY"),
    base_url="https://api.deepseek.com"
)
_ds_resp = _ds_client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": "Output only the number (e.g., 0, 1, 2)."},
        {"role": "user", "content": full_prompt}
    ]
)
deepseek_answer = _ds_resp.choices[0].message.content.strip()

print("=== Gloss Selection Results (number only expected) ===")
print(f"openai:gpt-4o -> {openai_answer}")
print(f"anthropic:claude-3-5-sonnet-20241022 -> {claude_answer}")
print(f"gemini:gemini-1.5-pro-latest -> {gemini_answer}")
print(f"deepseek:deepseek-chat -> {deepseek_answer}")

export OPENAI_API_KEY=sk-...

export ANTHROPIC_API_KEY=sk-ant-...

export GEMINI_API_KEY=AIza...

export DEEPSEEK_API_KEY=sk-deepseek-...

`python run_gloss_llms_min.py "Context: The bank of the river was steep. Options: 0) financial institution 1) sloping land 2) storage place"`
